# Introduction to Patient matching
This notebook introduces key concepts of patient matching while demonstrating those concepts using python

*********************

## About this notebook


* This notebook provides an introduction and demonstration of the patient matching process.This tool is intended for anyone who wants a better understand the matching process with a practical example. This noteboook uses syntehtic data but can also run with your own datasets.


* This tool uses the  **Python Record Linkage Toolkit** to demonstrate the patient matching process.The **Python Record Linkage Toolkit** is a library to link records in or between data sources. The toolkit provides
most of the tools needed for record linkage and deduplication. 

    * Link github: [Python Record Linkage Toolkit](https://github.com/J535D165/recordlinkage)
    * Link Documentation: [Documentation : Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/index.html#)

*  To use this notebook with your dataset you need to replace both dataset :deduplication and the gold standard file with your own data file. However the fields in your data files should be exact same as the ones in the notebook, if not you can still for each step comment the code corresponding to the fields not available.

## What is patient matching?

* Data matching: The process to identify records that refer to the same real-world entity within one or across several databases.
    * When applied on one database, this process is called deduplication.
    * Also known as entity resolution, object identification, duplicate detection.


**Patient matching**: Comparing data from multiple sources to identify records that represent the same patient.

## Python Record Linkage Toolkit

* The **Python Record Linkage Toolkit** is a library to link records in or between data sources. The toolkit provides
most of the tools needed for record linkage and deduplication. 
* The package contains indexing methods, functions to compare records and classifiers. 
* The project is inspired by the Freely Extensible Biomedical Record Linkage (FEBRL,Christen et al. 2002, 2004) project but add extensive use of data manipulation tools like pandas and numpy and machine learning tools like scikit-learn.
* Link github: [Python Record Linkage Toolkit](https://github.com/J535D165/recordlinkage)
* Link Documentation: [Documentation : Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/index.html#)

## Patient matching process (deduplication)

* Record linkage is a complex process and requires the user to understand, up to a certain degree, many technical details.
* The process consists of 5 steps : **Pre-processing**, **Indexing/Blocking**, **Comparison**,**Classification**, **Evaluation**
* It is important to understand the process because each step influence the matching results
* For example understanding blocking strategy is crucial because affecting performance and matching results


![record linkage process](./docs/matching_process_dedup.PNG "record linkage")

In [1]:
import recordlinkage  as rl
import numpy as np
import pandas as pd
import warnings
from recordlinkage.index import Block
from recordlinkage.datasets import load_febrl4,load_febrl3,load_febrl2,load_febrl1
from recordlinkage.preprocessing import phonetic
warnings.filterwarnings('ignore')

#### Get version information

In [2]:
# Get Version information
print("Pandas version: {0}".format(pd.__version__),'\n')
print("Python Record Linkage version: {0}".format(rl._version.get_versions()['version']),'\n')
print("Numpy version: {0}".format(np.__version__),'\n')

Pandas version: 1.5.3 

Python Record Linkage version: 0.15 

Numpy version: 1.20.3 



##  Importing dataset to deduplicate
Before we start with the deduplication process first we import the data in notebook. We assume the data has been cleaned and mapped to be imported in the format below :

| Variable| Description | Format |
| --- | --- | --- |
| rec_id | Internal ID in your data | |
| given_name | patient first name | |
| surname | patient last name | |
| date_of_birth | patient date of birth | YYYYMMDD  |
| address_1 | patient address first part | |
| address_2 | patient address second part | |
| suburb | patient address surburb | |
| postcode | patient postal code | |
| state | patient address state | |
| soc_soc_id | social security number | |

In this notebook we use  synthetic dataset from The Python Record Linkage Toolkit (PRLT). The PRLT contains several open public synthetic datasets. The package is distributed with a  four synthetic datasets. For this project we will use The Freely Extensible Biomedical Record Linkage (Febrl) dataset 3 . **Dataset 3 (FEBRL3) contains 5000 records (2000 originals and 3000 duplicates)**.


In [3]:
# file to deduplicate
IMPORT_FILE_TO_DEDUPLICATE = './data/dataset_febrl3.csv'
df_a = pd.read_csv(IMPORT_FILE_TO_DEDUPLICATE)
df_a = df_a.set_index('rec_id')

print("Total number of records:", len(df_a))
df_a.sort_values('surname').head()

Total number of records: 5000


given_name surname      address_1                 address_2  \
rec_id                                                                       
rec-1307-dup-2   campbell   abbey   adair street                   arcadai   
rec-515-dup-3        tyna    abea  haines street                       NaN   
rec-515-dup-0       tynna   abera  haines street                    eureka   
rec-515-dup-1       tynan   abera   hainesstreet                       NaN   
rec-515-dup-2       tynan   abera  haines street  st john of go d hospital   

                  suburb  postcode state  date_of_birth  soc_sec_id  
rec_id                                                               
rec-1307-dup-2  wantirna      6051    sa     19211108.0     1327917  
rec-515-dup-3   ballarat      2303   nsw     19370412.0     5824612  
rec-515-dup-0   ballarat      2303   nws     19370412.0     5824612  
rec-515-dup-1   ballarat      2303   nws     19370412.0     6873174  
rec-515-dup-2   ballarat      2033   nsw     19370412.0     5821612

For more info : [Synthetic datasets](https://recordlinkage.readthedocs.io/en/latest/ref-datasets.html "Title")

## Pre-Processing and Standardization

The main task of data cleaning and standardization is the conversion of the raw input data into well defined, consistent forms, as well as the resolution of inconsistencies in the way information is represented and encoded.(Christen 2012)
* Cleaning patient names,  addresses and hospital names
* Parsing  patient names, addresses and phone number
* Formatting and coding dates (e.g. date of birth, date of ART initiation)

We have split the date of birth in 3 columns for more easy comparison, also we have calculated the metaphone of the given name and surname respectively.

**Metaphone** is a phonetic encoding algorithm used to encode the way words an syllable are pronounces to help **reduce minor typographical error**.The output of a phonetic algorithm is an intentionally approximate phonetic representation of the word. With application still limited to English words Metaphone is an improvement on the Soundex algorihtm .


In [4]:
df_a['date_of_birth'] = pd.to_datetime(df_a['date_of_birth'],format='%Y%m%d', errors='coerce')
#df_a['date_of_birth'] = df_a['date_of_birth'].astype(str)
df_a['YearB'] = df_a['date_of_birth'].dt.year.astype('Int64') 
df_a['MonthB'] = df_a['date_of_birth'].dt.month.astype('Int64') 
df_a['DayB'] = df_a['date_of_birth'].dt.day.astype('Int64')
#df_a['YearB'] = df_a['date_of_birth'].str[:4].astype(str)
#df_a['MonthB'] = df_a['date_of_birth'].str[5:7].astype(str)
#df_a['DayB'] = df_a['date_of_birth'].str[6:].astype(str)

df_a['metaphone_given_name'] = phonetic(df_a['given_name'], method='metaphone')
df_a['metaphone_surname'] = phonetic(df_a['surname'], method='metaphone')
df_a.sort_values('surname').head() 

given_name surname      address_1                 address_2  \
rec_id                                                                       
rec-1307-dup-2   campbell   abbey   adair street                   arcadai   
rec-515-dup-3        tyna    abea  haines street                       NaN   
rec-515-dup-0       tynna   abera  haines street                    eureka   
rec-515-dup-1       tynan   abera   hainesstreet                       NaN   
rec-515-dup-2       tynan   abera  haines street  st john of go d hospital   

                  suburb  postcode state date_of_birth  soc_sec_id  YearB  \
rec_id                                                                      
rec-1307-dup-2  wantirna      6051    sa    1921-11-08     1327917   1921   
rec-515-dup-3   ballarat      2303   nsw    1937-04-12     5824612   1937   
rec-515-dup-0   ballarat      2303   nws    1937-04-12     5824612   1937   
rec-515-dup-1   ballarat      2303   nws    1937-04-12     6873174   1937   
rec-515-dup-2   ballarat      2033   nsw    1937-04-12     5821612   1937   

                MonthB  DayB metaphone_given_name metaphone_surname  
rec_id                                                               
rec-1307-dup-2      11     8                KMPBL                AB  
rec-515-dup-3        4    12                   TN                AB  
rec-515-dup-0        4    12                   TN               ABR  
rec-515-dup-1        4    12                  TNN               ABR  
rec-515-dup-2        4    12                  TNN               ABR

## Blocking 

The second step of the process called **blocking or indexing** try to reduce the number of records we need to compare. The idea is instead of comparing all records of the dataset between themselves we want to compare only the records that are most likely to be matched. 


### Blocking Key

#### Phonetic encoding

Phonetic encoding convert a string name into a code according to how it is pronounced. This help reduce minor typographical errors.


In [5]:
from jellyfish import soundex, metaphone
from metaphone import doublemetaphone


print('Soundex')
print('*'*50)
print("mayer:",soundex('mayer'))
print("meyer :",soundex('meyer'))
print("myer :",soundex('myer'))
print("mayar :",soundex('mayar'))
print("mayor :",soundex('mayor'))
print("maier :",soundex('maier'))


print('Metaphone')
print('*'*50)
print("mayer:",metaphone('mayer'))
print("meyer :",metaphone('meyer'))
print("myer :",metaphone('myer'))
print("mayar :",metaphone('mayar'))
print("mayor :",metaphone('mayor'))
print("maier :",metaphone('maier'))

print('DoubleMetaphone')
print('*'*50)
print("mayer:",doublemetaphone('mayer'))
print("meyer :",doublemetaphone('meyer'))
print("myer :",doublemetaphone('myer'))
print("mayar :",doublemetaphone('mayar'))
print("mayor :",doublemetaphone('mayor'))
print("maier :",doublemetaphone('maier'))

ModuleNotFoundError: No module named 'metaphone'

**In PRLT Phonetic encoding possible options are “soundex”, “nysiis”, “metaphone” or “match_rating”.**

**Others not included : double-metaphne, phonix , phonex, OCNA, Fuzzy soundex (Christen 2012)**

#### Full Indexing

##### Comparing each record to all other records pairs.

In [ ]:
indexer = rl.Index()
indexer.full()
candidate_record_pairs = indexer.index(df_a)
print("Number of candidate record pairs", len(candidate_record_pairs))
#candidate_record_pairs.to_frame(index=False)

#(5000*5000-5000)/2 = 12 497 500

Number of candidate record pairs 12497500


#### Blocking Key

##### Example 1) Only compare records with the same last name

In [ ]:
indexer = rl.Index()
indexer.add(Block(['surname']))
candidate_record_pairs = indexer.index(df_a)
print("Number of candidate record pairs :", len(candidate_record_pairs))
#candidate_record_pairs.to_frame(index=False)
df_a_1 = df_a[['given_name','surname','date_of_birth']]

df_c = candidate_record_pairs.to_frame(index=False)
df_c.join(df_a_1,on=['rec_id_1']).join(df_a_1,on=['rec_id_2'], lsuffix='_1', rsuffix='_2').head(20)


Number of candidate record pairs : 37255


rec_id_1        rec_id_2 given_name_1 surname_1 date_of_birth_1  \
0   rec-1191-dup-3    rec-1496-org         loan     green      1944-05-14   
1     rec-1076-org    rec-1496-org        sarah     green      1914-09-08   
2     rec-1076-org  rec-1191-dup-3        sarah     green      1914-09-08   
3     rec-1297-org    rec-1496-org       emiily     green      1938-06-25   
4     rec-1297-org  rec-1191-dup-3       emiily     green      1938-06-25   
5     rec-1297-org    rec-1076-org       emiily     green      1938-06-25   
6     rec-1357-org    rec-1496-org       jordan     green             NaT   
7     rec-1357-org  rec-1191-dup-3       jordan     green             NaT   
8     rec-1357-org    rec-1076-org       jordan     green             NaT   
9     rec-1357-org    rec-1297-org       jordan     green             NaT   
10     rec-759-org    rec-1496-org       samuel     green      1902-06-01   
11     rec-759-org  rec-1191-dup-3       samuel     green      1902-06-01   
12     rec-759-org    rec-1076-org       samuel     green      1902-06-01   
13     rec-759-org    rec-1297-org       samuel     green      1902-06-01   
14     rec-759-org    rec-1357-org       samuel     green      1902-06-01   
15    rec-1004-org    rec-1496-org        tayla     green      1931-07-15   
16    rec-1004-org  rec-1191-dup-3        tayla     green      1931-07-15   
17    rec-1004-org    rec-1076-org        tayla     green      1931-07-15   
18    rec-1004-org    rec-1297-org        tayla     green      1931-07-15   
19    rec-1004-org    rec-1357-org        tayla     green      1931-07-15   

   given_name_2 surname_2 date_of_birth_2  
0      mitchell     green      1956-04-09  
1      mitchell     green      1956-04-09  
2          loan     green      1944-05-14  
3      mitchell     green      1956-04-09  
4          loan     green      1944-05-14  
5         sarah     green      1914-09-08  
6      mitchell     green      1956-04-09  
7          loan     green      1944-05-14  
8         sarah     green      1914-09-08  
9        emiily     green      1938-06-25  
10     mitchell     green      1956-04-09  
11         loan     green      1944-05-14  
12        sarah     green      1914-09-08  
13       emiily     green      1938-06-25  
14       jordan     green             NaT  
15     mitchell     green      1956-04-09  
16         loan     green      1944-05-14  
17        sarah     green      1914-09-08  
18       emiily     green      1938-06-25  
19       jordan     green             NaT

##### Example 2) Only compare records with the same  metaphone of last name

In [ ]:
indexer = rl.Index()
indexer.add(Block(['metaphone_surname']))
candidate_record_pairs = indexer.index(df_a)
print("Number of candidate record pairs :", len(candidate_record_pairs))
#candidate_record_pairs.to_frame(index=False)
df_a_1 = df_a[['given_name','surname','date_of_birth','metaphone_surname']]

df_c = candidate_record_pairs.to_frame(index=False)
df_c.join(df_a_1,on=['rec_id_1']).join(df_a_1,on=['rec_id_2'], lsuffix='_1', rsuffix='_2').head()

Number of candidate record pairs : 45229


rec_id_1        rec_id_2 given_name_1 surname_1 date_of_birth_1  \
0  rec-1191-dup-3    rec-1496-org         loan     green      1944-05-14   
1     rec-686-org    rec-1496-org         toby    garran      1952-08-17   
2     rec-686-org  rec-1191-dup-3         toby    garran      1952-08-17   
3    rec-1076-org    rec-1496-org        sarah     green      1914-09-08   
4    rec-1076-org  rec-1191-dup-3        sarah     green      1914-09-08   

  metaphone_surname_1 given_name_2 surname_2 date_of_birth_2  \
0                 KRN     mitchell     green      1956-04-09   
1                 KRN     mitchell     green      1956-04-09   
2                 KRN         loan     green      1944-05-14   
3                 KRN     mitchell     green      1956-04-09   
4                 KRN         loan     green      1944-05-14   

  metaphone_surname_2  
0                 KRN  
1                 KRN  
2                 KRN  
3                 KRN  
4                 KRN

As example you can decide to compare only patiend with the same : first name, last name and date of birth. This combination of fields is called a **  blocking key**. Using  a blocking key provide a reduce set of record pairs. In this notebook we use multiple blocking keys and consider the  ** union  ** of all the results set of candidate record pairs to evaluate for matching in the next steps.

In other terms we compare patient with :
1. (the same **'metaphone_given_name','metaphone_surname','date_of_birth'**) UNION 
2. (the same **'metaphone_given_name','DayB'**) UNION 
3. (the same **'metaphone_given_name','MonthB'**) UNION 
4. (the same **date_of_birth**) UNION 
5. (the same **'metaphone_surname','YearB'**)


Please note the use of the **metaphone** algorithm here instead of the exact value.This takes into account typrographic errors in the names and provide a wider range of candidiate record pairs

In [ ]:
indexer = rl.Index()

# describe each step in the union

# soundex firstname, methapone surname, exact date of birth
indexer.add(Block(['metaphone_given_name','metaphone_surname','date_of_birth']))
# soundex firstname , day of birth
indexer.add(Block(['metaphone_given_name','DayB']))
#soundex firstname , month of birth
indexer.add(Block(['metaphone_given_name','MonthB']))
# metaphone surname, year of birth 
indexer.add(Block(['metaphone_surname','YearB']))
# ssn
indexer.add(Block(['soc_sec_id']))


candidate_record_pairs = indexer.index(df_a)
print("Number of candidate record pairs :", len(candidate_record_pairs))
#candidate_record_pairs.to_frame(index=False)
df_a_1 = df_a[['given_name','surname','date_of_birth','metaphone_surname','metaphone_given_name','soc_sec_id']]

df_c = candidate_record_pairs.to_frame(index=False)
df_c.join(df_a_1,on=['rec_id_1']).join(df_a_1,on=['rec_id_2'], lsuffix='_1', rsuffix='_2').head()

Number of candidate record pairs : 12833


rec_id_1        rec_id_2 given_name_1 surname_1 date_of_birth_1  \
0  rec-0-org    rec-1023-org        jinni    dreyer      1942-01-27   
1  rec-0-org  rec-1540-dup-1        jinni    dreyer      1942-01-27   
2  rec-1-org    rec-1643-org        jonah    browne      1925-04-13   
3  rec-1-org    rec-1986-org        jonah    browne      1925-04-13   
4  rec-1-org      rec-41-org        jonah    browne      1925-04-13   

  metaphone_surname_1 metaphone_given_name_1  soc_sec_id_1 given_name_2  \
0                TRYR                     JN       3787407       gianni   
1                TRYR                     JN       3787407         john   
2                 BRN                     JN       8328406       joanna   
3                 BRN                     JN       8328406        jonah   
4                 BRN                     JN       8328406       brodee   

  surname_2 date_of_birth_2 metaphone_surname_2 metaphone_given_name_2  \
0    matson      1941-01-11                MTSN                     JN   
1    benger      1971-01-26                BNJR                     JN   
2    dooley      1993-05-13                  TL                     JN   
3    thorpe      1958-04-20                 0RP                     JN   
4     brain      1925-06-23                 BRN                    BRT   

   soc_sec_id_2  
0       2540080  
1       5651019  
2       4843681  
3       6699959  
4       3821503

## Comparison

Identifying the similarity between records pairs to create a comparison vectors. 

The previous step provided us a list of record pairs. In this step we compare the corresponding fields of each record pair using string distance algorithm.

**Comparison function** :
A function that has input two atrtributes values (string,number, dates, time,complex objects) and calculate the similarity between these two values. The comparison can be exact or  allow for approximate similarity.An exact returns 0 or 1 and approximate returns a normalized value between zero and 1.

### String Comparison function or String distance algorithm

  - Edit distance
  - Jaccard distance
  - SequenceMatcher
  - Levenshtein
  - Jaro
  - Jaro-Winkler
  - Jaccard similarity
  - SoftTf-Idf


In [ ]:
from jellyfish import jaro_winkler,levenshtein_distance,hamming_distance,jaro_distance

print("*****************jaro_distance***********************")
print("mayer vs meyer :",jaro_distance('mayer','meyer'))
print("mayer vs myer :",jaro_distance('mayer','myer'))
print("mayer vs mayar :",jaro_distance('mayer','mayar'))
print("mayer vs mayor :",jaro_distance('mayer','mayor'))
print("mayer vs maier :",jaro_distance('mayer','maier'))


print("*****************jaro_winkler***********************")
print("mayer vs meyer :",jaro_winkler('mayer','meyer'))
print("mayer vs myer :",jaro_winkler('mayer','myer'))
print("mayer vs mayar :",jaro_winkler('mayer','mayar'))
print("mayer vs mayor :",jaro_winkler('mayer','mayor'))
print("mayer vs maier :",jaro_winkler('mayer','maier'))


print("*************levenshtein_distance ************************")
print("mayer vs meyer :",levenshtein_distance('mayer','meyer'))
print("mayer vs myer :",levenshtein_distance('mayer','myer'))
print("mayer vs mayar :",levenshtein_distance('mayer','mayar'))
print("mayer vs mayor :",levenshtein_distance('mayer','mayor'))
print("mayer vs maier :",levenshtein_distance('mayer','maier'))



*****************jaro_distance***********************
mayer vs meyer : 0.8666666666666667
mayer vs myer : 0.9333333333333332
mayer vs mayar : 0.8666666666666667
mayer vs mayor : 0.8666666666666667
mayer vs maier : 0.8666666666666667
*****************jaro_winkler***********************
mayer vs meyer : 0.88
mayer vs myer : 0.94
mayer vs mayar : 0.9066666666666667
mayer vs mayor : 0.9066666666666667
mayer vs maier : 0.8933333333333333
*************levenshtein_distance ************************
mayer vs meyer : 1
mayer vs myer : 1
mayer vs mayar : 1
mayer vs mayor : 1
mayer vs maier : 1


The Python Record Linkage Toolkit implemented algorithms are: **‘jaro’**,**’jarowinkler’**, **‘levenshtein’**, **‘damerau_levenshtein’**, **‘qgram’** or **‘cosine’**. 

### Complex aggreement pattern

To create a complex aggreement pattern,for each record pair we compare the corresponding fields using a string comparison or string distance function
 
| Variable| comparison function |
| --- | --- |
| given_name | jarowinkler| 
| surname | jarowinkler |
| date_of_birth | exact matching |
| soc_sec_id | exact matching|
| address_1 | levenshtein |
| address_2 | levenshtein |
| suburb | levenshtein |
| postcode | exact matching |
| state | exact matching |

The output of the comparison is the **comparison vector** that will be used for classification.

In [ ]:
compare_cl = rl.Compare()
compare_cl.string('given_name', 'given_name', method='jarowinkler', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler',label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('soc_sec_id', 'soc_sec_id', label='soc_sec_id')
compare_cl.string('address_1', 'address_1', method ='levenshtein' , label='address_1')
compare_cl.string('address_2', 'address_2', method ='levenshtein' , label='address_2')
compare_cl.string('suburb', 'suburb', method ='levenshtein', label='suburb')
compare_cl.exact('postcode', 'postcode', label='postcode')
compare_cl.exact('state', 'state', label='state')

features = compare_cl.compute(candidate_record_pairs, df_a)
features.head()
#print("Number of candidate record pairs :", len(features))

given_name   surname  date_of_birth  soc_sec_id  \
rec_id_1  rec_id_2                                                          
rec-0-org rec-1023-org      0.822222  0.000000              0           0   
          rec-1540-dup-1    0.633333  0.666667              0           0   
rec-1-org rec-1643-org      0.791111  0.555556              0           0   
          rec-1986-org      1.000000  0.555556              0           0   
          rec-41-org        0.455556  0.760000              0           0   

                          address_1  address_2    suburb  postcode  state  
rec_id_1  rec_id_2                                                         
rec-0-org rec-1023-org     0.615385   0.142857  0.133333         0      0  
          rec-1540-dup-1   0.529412   0.142857  0.133333         0      1  
rec-1-org rec-1643-org     0.250000   0.000000  0.000000         0      0  
          rec-1986-org     0.312500   0.000000  0.142857         0      0  
          rec-41-org       0.312500   0.000000  0.000000         0      0

### Simple (Binary) aggreement pattern

 To continue our example we will use a simple aggreement pattern.
 We will use a threshold for the string comparison function - if the **score >0.85** we say there's agreeement (1) if not there's disagreement (0).
 
| Variable| comparison function | threshold |
| --- | --- |---|
| given_name | jarowinkler| 0.85 |
| surname | jarowinkler | 0.85
| date_of_birth | exact matching |
| soc_sec_id | exact matching||
| address_1 | levenshtein | 0.85|
| address_2 | levenshtein | 0.85 |
| suburb | levenshtein | 0.85 |
| postcode | exact matching | |
| state | exact matching | |

The output of the comparison is the **comparison vector** that will be used for classification.

In [ ]:
compare_cl = rl.Compare()
compare_cl.string('given_name', 'given_name', method='jarowinkler', threshold = 0.85, label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler',threshold = 0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('soc_sec_id', 'soc_sec_id', label='soc_sec_id')
compare_cl.string('address_1', 'address_1', method ='levenshtein' ,threshold = 0.85, label='address_1')
compare_cl.string('address_2', 'address_2', method ='levenshtein' ,threshold = 0.85, label='address_2')
compare_cl.string('suburb', 'suburb', method ='levenshtein' ,threshold = 0.85, label='suburb')
compare_cl.exact('postcode', 'postcode', label='postcode')
compare_cl.exact('state', 'state', label='state')

features = compare_cl.compute(candidate_record_pairs, df_a)
features.head()
#print("Number of candidate record pairs :", len(features))

given_name  surname  date_of_birth  soc_sec_id  \
rec_id_1  rec_id_2                                                         
rec-0-org rec-1023-org           0.0      0.0              0           0   
          rec-1540-dup-1         0.0      0.0              0           0   
rec-1-org rec-1643-org           0.0      0.0              0           0   
          rec-1986-org           1.0      0.0              0           0   
          rec-41-org             0.0      0.0              0           0   

                          address_1  address_2  suburb  postcode  state  
rec_id_1  rec_id_2                                                       
rec-0-org rec-1023-org          0.0        0.0     0.0         0      0  
          rec-1540-dup-1        0.0        0.0     0.0         0      1  
rec-1-org rec-1643-org          0.0        0.0     0.0         0      0  
          rec-1986-org          0.0        0.0     0.0         0      0  
          rec-41-org            0.0        0.0     0.0         0      0

## Classification

Based on comparison results, this step uses a classification algorithm to classify candidate records pairs in: matches, non-matches or potential matches.

Probabilistic matching is based on a probability model that designates record pairs as matches, possible matches, or non-matches based on calculation of linkage scores and application of decision rules about these scores to define true matches. 

Options:
 - Simple Scoring
 - Rules
 - Probabilistic matching
 - Supervised learning

### An example of simple score-based or threshold-based matching

The simplest way to classisfy  candidate records pairs into two classes matches and non-matches is to sum the similarity values in their comparison vector into a single total similarity value and apply a similarity **threshold**  to decide in which class a record pair belongs.

This is what we did in the example below, adding a column **simsum**, we select a **threshold = 4**. All record pairs with s similarity sum greater or equal to 4 are considered as matches.

Reference:
* Christen, Peter. 2012. Data matching: concepts and techniques for record linkage, entity resolution, and duplicate detection. Springer Science & Business Media.

In [ ]:
df_f = features.copy()
df_f['simsum'] = features.sum(axis=1)

threshold = 4 

#threshold or score based classification
matches = df_f[df_f['simsum'] >=threshold]
print("Number of match record pairs :", len(matches))
matches.head(20)

Number of match record pairs : 6258


given_name  surname  date_of_birth  soc_sec_id  \
rec_id_1       rec_id_2                                                         
rec-10-dup-0   rec-10-dup-2           0.0      0.0              1           1   
rec-10-dup-1   rec-10-dup-0           0.0      0.0              1           1   
               rec-10-dup-2           1.0      1.0              1           1   
rec-10-org     rec-10-dup-0           0.0      0.0              1           1   
               rec-10-dup-1           1.0      1.0              1           1   
               rec-10-dup-2           1.0      1.0              1           1   
rec-100-dup-0  rec-100-dup-1          1.0      1.0              1           1   
               rec-100-dup-3          1.0      1.0              1           1   
rec-100-dup-2  rec-100-dup-0          1.0      1.0              0           1   
               rec-100-dup-1          1.0      1.0              0           1   
               rec-100-dup-3          1.0      1.0              0           1   
               rec-100-org            1.0      1.0              0           1   
rec-100-dup-3  rec-100-dup-1          1.0      1.0              1           1   
rec-100-org    rec-100-dup-0          1.0      1.0              1           1   
               rec-100-dup-1          1.0      1.0              1           1   
               rec-100-dup-3          1.0      1.0              1           1   
rec-1000-dup-1 rec-1000-dup-0         1.0      1.0              1           1   
               rec-1000-org           1.0      1.0              1           1   
rec-1000-org   rec-1000-dup-0         1.0      1.0              1           1   
rec-1001-dup-0 rec-1001-org           0.0      1.0              0           1   

                               address_1  address_2  suburb  postcode  state  \
rec_id_1       rec_id_2                                                        
rec-10-dup-0   rec-10-dup-2          0.0        1.0     1.0         0      1   
rec-10-dup-1   rec-10-dup-0          0.0        1.0     1.0         0      1   
               rec-10-dup-2          0.0        1.0     1.0         1      1   
rec-10-org     rec-10-dup-0          1.0        1.0     1.0         0      1   
               rec-10-dup-1          0.0        1.0     1.0         1      1   
               rec-10-dup-2          0.0        1.0     1.0         1      1   
rec-100-dup-0  rec-100-dup-1         0.0        1.0     1.0         1      1   
               rec-100-dup-3         0.0        1.0     1.0         1      1   
rec-100-dup-2  rec-100-dup-0         0.0        1.0     1.0         1      1   
               rec-100-dup-1         1.0        1.0     1.0         1      1   
               rec-100-dup-3         1.0        1.0     1.0         1      1   
               rec-100-org           1.0        1.0     1.0         1      1   
rec-100-dup-3  rec-100-dup-1         0.0        1.0     1.0         1      1   
rec-100-org    rec-100-dup-0         0.0        1.0     1.0         1      1   
               rec-100-dup-1         1.0        1.0     1.0         1      1   
               rec-100-dup-3         1.0        1.0     1.0         1      1   
rec-1000-dup-1 rec-1000-dup-0        0.0        0.0     1.0         1      1   
               rec-1000-org          0.0        0.0     1.0         1      1   
rec-1000-org   rec-1000-dup-0        0.0        1.0     1.0         1      1   
rec-1001-dup-0 rec-1001-org          1.0        1.0     1.0         1      1   

                               simsum  
rec_id_1       rec_id_2                
rec-10-dup-0   rec-10-dup-2       5.0  
rec-10-dup-1   rec-10-dup-0       5.0  
               rec-10-dup-2       8.0  
rec-10-org     rec-10-dup-0       6.0  
               rec-10-dup-1       8.0  
               rec-10-dup-2       8.0  
rec-100-dup-0  rec-100-dup-1      8.0  
               rec-100-dup-3      8.0  
rec-100-dup-2  rec-100-dup-0      7.0  
               rec-100-dup-1      8.0  
       

## Evaluation

Comparing match results with the known ground truth or gold standard to mesaure the performance of the matching process.


### Gold standard 
The main objective of evaluation techniques is to achieve **high matching quality** in order to assess  the quality of the matched  data for a certain project ground-truth data also known as gold standard is are required.

There are several approches of how ground-thruth data can be generated. In this notebook the gold standard data was generated as part of the synthetic data used for matching.

In [ ]:
# gold_ standard or known truth
IMPORT_FILE_GOLD_STANDARD = './data/dataset_febrl3_true_links.csv'
df_true_links = pd.read_csv(IMPORT_FILE_GOLD_STANDARD)
df_true_links.set_index(['rec_id_1','rec_id_2'],inplace=True)
df_true_links.head()

Empty DataFrame
Columns: []
Index: [(rec-552-dup-1, rec-552-dup-3), (rec-552-dup-0, rec-552-dup-3), (rec-552-dup-0, rec-552-dup-1), (rec-552-org, rec-552-dup-3), (rec-552-org, rec-552-dup-1)]

### Confusion matrix

Assuming you have the ground-truth (i.e true match status of record pairs) and matching results from a project you can assigned each record pairs into one of the following categories :

* **False Positives (FP)**: Also referred to as a Type I error, refers to a classification error by the matching algorithm where a record pair is marked as a match but in reality the two records refer to two distinct patients.


* **True Positives (TP)**: Refers to the correct classification by the matching algorithm of two patient records as a match when both records refer to the same person.


* **False Negatives (FN)**: Also referred to as a Type II error, refers to a classification error by the matching algorithm where the two patient records are marked as referring to two distinct patients but in reality the two records refer to the same person.


* **True Negatives (TN)**: Refers to the correct classification by the matching algorithm of two patient records as a non-match when the two records refer to two different patients

This is represented in the table below known as the **confusion matrix**:

![confusion matrix](./docs/confusion_matrix.png "record linkage")

In [ ]:
matrix  = rl.confusion_matrix(df_true_links, matches, len(features))
matrix

array([[6258,  280],
       [   0, 6295]])

### Evaluation metrics
They are commonly used to evaluate the performance of matching algorithms and configuration changes to those algorithms. 

**Precision** :
A measure of the fraction of pairs that the matching algorithm classified accurately as matches. It is also called the positive predictor value and can be used along with the sensitivity to jointly evaluate the performance of a matching algorithm.

\begin{align*}
  Precision &= \frac{TP}{TP+FP}\\
\end{align*}


In [ ]:
rl.precision(df_true_links, matches)

1.0

**Recall** :
Also referred to as **sensistivity**, is a measure of the capability of the matching algorithm to correctly classify two records that refer to the same patient as true matches. It is calculated as the ratio of the number of true positives divided by the sum of true positives and false negatives.

\begin{align*}
  Recall &= \frac{TP}{TP+FN}\\
\end{align*}


In [ ]:
rl.recall(df_true_links, matches)

0.9571734475374732

**F-Score** :
Represents the harmonic mean of the Precision, and Recall, which is not influenced by the large number of true non matches.

\begin{align*}
  Fscore &= \frac{2 x Precision x Recall}{Precision+Recall}\\
\end{align*}

In [ ]:
# The F-score for this classification is
rl.fscore(df_true_links, matches)

0.9781181619256017

## References

* de Bruin, Jonathan. 2017. “Record Linkage. Python library. Version 0.8.1.” https://recordlinkage.readthedocs.io/.

* Herzog, Thomas N, Fritz J Scheuren and William E Winkler. 2007. Data quality and record linkage techniques. Vol. 1 Springer.

* Fellegi, Ivan P and Alan B Sunter. 1969. “A theory for record linkage.” Journal of the American Statistical Association 64(328):1183–1210.

* Christen, Peter. 2012. Data matching: concepts and techniques for record linkage, entity resolution, and duplicate detection. Springer Science & Business Media.